In [2]:
import requestcompletion as rc
from pydantic import BaseModel, Field
from copy import deepcopy

In [20]:
gpt_4o = rc.llm.OpenAILLM("gpt-4o")

class AnswerPrediction(BaseModel):
    quality_answer: bool = Field(..., description="The quality of the answer that was provided by the model.")

system_c_o_t = rc.llm.SystemMessage("You are extremley intellegnt reviewer of information that ponders the provided question and context to provide a simple one paragraph plan where you discuss you thinking process and how you would go about answering the question, without explictely answer the question.")
system_answer = rc.llm.SystemMessage("You are a helpful assistant that works tirelessly to answer a question using the provided context as a guide to answer the question.")
system_answer_reviewer = rc.llm.SystemMessage("You are a reviewer of the plan that was provided by the model and will determine if the plan is feasible and if it will accomplish the task that was asked of you. Be harsh and clear when the plan is not adequete.")



COTNode = rc.library.terminal_llm("COT", system_message=system_c_o_t, model=gpt_4o)
AnswerNode = rc.library.terminal_llm("Answerer", system_message=system_answer, model=gpt_4o)
ReviewerNode = rc.library.structured_llm(pretty_name="AnswerReviewer", output_model=AnswerPrediction, system_message=system_answer_reviewer, model=gpt_4o)



async def COTLLM(message_history: rc.llm.MessageHistory, number_trails: int = 4):
    original_message_history = deepcopy(message_history)
    for _ in range(number_trails):
        cot_response = await rc.call(COTNode, message_history=message_history)

        message_history.append(rc.llm.AssistantMessage("My plan:" + cot_response))

        answer_response = await rc.call(ReviewerNode, message_history=message_history)
        if not answer_response.quality_answer:
            continue
        else:
            break
    
    original_message_history.append(rc.llm.UserMessage("Contextual Plan: " + cot_response))
    response = await rc.call(AnswerNode, message_history=original_message_history)

    return cot_response, response

    
ChainOfThought = rc.library.from_function(COTLLM)




In [ ]:
message_history = rc.llm.MessageHistory([rc.llm.UserMessage("")])
with rc.Runner() as runner:
    response = await runner.run(ChainOfThought, message_history=message_history)

print("COT plan: " + response.answer[0])
print("-------------------")
print("Answer: " + response.answer[1])

c:\Users\Logan\Documents\rc\.venv\lib\site-packages\requestcompletion\nodes\library\terminal_llm.py:22: UserWarning: System message already exists in message history. We will replace it.
  warnings.warn("System message already exists in message history. We will replace it.")


COT plan: To address the question of how to make a cake, I would start by considering the fundamental components and steps involved in cake baking. First, I would identify the basic ingredients typically required, such as flour, sugar, eggs, butter, and a leavening agent like baking powder or baking soda. Next, I would explore the process of combining these ingredients, which usually involves creaming the butter and sugar, adding eggs, and then alternating between adding dry and wet ingredients. I would also consider the importance of preheating the oven to the correct temperature and preparing the baking pan to prevent sticking. Additionally, I would think about the different types of cakes, such as sponge, butter, or chiffon, as each might have specific techniques or variations in the method. Finally, I would reflect on the cooling and decorating process, which can be crucial for the final presentation and taste. By organizing these thoughts, I would be able to construct a clear and 